In [2]:
import numpy as np
from id3 import ID3DecisionTree

In [4]:
# Sample data
y =  np.random.choice([1], size=9)

In [10]:
play_y_1 = np.full(shape=(9,), fill_value=1)
play_y_0 = np.full(shape=(5,), fill_value=0)
play_y = np.concatenate((play_y_1, play_y_0))

In [11]:
# Instantiate tree
tree = ID3DecisionTree(x=None, y=play_y)

In [12]:
tree.compute_entropy_of_set_s()

0.9402859586706309